BVP

laplace equation
concentric balls around center (0, 0) - drichlet bc on both the surfaces
smallest eigenvalues and functions

In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
import pyvista as pv
from dolfinx import mesh, fem, plot

#1 generate concentric balls domain of radius 3 and 1

In [2]:
import gmsh
from dolfinx.io import gmshio
# import meshio


def get_domain():
    """
    radius: float
    """

    # if filename is not None:
        
        


    gmsh.initialize()
    model = gmsh.model()
    model.add("domain")
    model.setCurrent("domain")

    rect = model.occ.addRectangle(0, 0, 0, 2, 2)
    # disk = model.occ.addDisk(0, 0, 0, disk_radius, disk_radius)

    model.occ.synchronize()

    #define boundaries

    #tag inner and outer surface
    # boundary = model.getBoundary(disk)
    # print(boundary)
    # outer_surface = boundary[0][1]
    # inner_surface = boundary[1][1]

    #add physical group
    model.addPhysicalGroup(2, [rect], tag = 1)
    # model.addPhysicalGroup(2, [inner_surface], tag = 2)
    # model.addPhysicalGroup(3, [shell_dims_tags[0][1]], tag = 3) #volume
    

    #set mesh options and generate mesh
    gmsh.option.setNumber("Mesh.MeshSizeFactor", 0.1)
    model.geo.synchronize()
    model.mesh.generate(2)
    gmsh.fltk.run()
    # gmsh.write("spherical_shell.msh")  # Export to file
    


    #get domain from mesh
    domain, cell_markers, facet_markers = gmshio.model_to_mesh(model=model, comm=MPI.COMM_WORLD, rank = 0, gdim = 2)

    
    
    # gmsh.write("Sphere.xmdf")

    gmsh.finalize()
    return domain, cell_markers, facet_markers

In [3]:
domain, cell_markers, facet_markers = get_domain()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000907728s, CPU 0.001312s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.199482s, CPU 0.193877s)
Info    : 5993 nodes 11988 elements


X_ChangeProperty: BadValue (integer parameter out of range for operation) 0x0


-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20241016
Build host    : 2976249cf102
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.9
OCC version   : 7.8.1
Packaged by   : conda
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [4]:
# calculate_volume(domain, cell_markers)


In [5]:
V = fem.functionspace(domain, ("Lagrange", 1))

In [7]:
facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, boundary)


NameError: name 'boundary' is not defined

In [ ]:

dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_markers)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'MeshTags'

#2 Setup function space and apply boundary conditions

In [ ]:
from ufl import Measure
def setup_fe_space(domain):
    """Create function space and variational forms. Defining bilinear and linear forms."""
    
    ds_measure = Measure("ds", domain=domain, subdomain_data=facet_markers)
    # dx_measure = Measure("dx", domain=domain) # Using default dx for the domain integral
    print(facet_markers)
    u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
    a = fem.form(ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx)
    m = fem.form(ufl.inner(u, v)*ds_measure)

    return V, a, m


In [9]:
V, a, m = setup_fe_space(domain)

preparing boundary conditions
Drichlet BC u = 0

In [10]:
from petsc4py import PETSc
def on_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), disk_radius)

def get_bcs(V):
    boundary_dofs = fem.locate_dofs_geometrical(V, on_boundary)
    bc = fem.dirichletbc(PETSc.ScalarType(0), boundary_dofs, V)

    return bc


#3 assemble matrices and apply boundary conditions

In [11]:
import dolfinx.fem.petsc

def assemble_and_apply_bc(V, a, m):

    #get bcs

    bc = get_bcs(V)

    #  Assemble matrices A and M

    A = fem.petsc.assemble_matrix(a, bcs=[]) #bc only on stiffness matrix
    A.assemble()
    M = fem.petsc.assemble_matrix(m) # mass matrix
    M.assemble()

    # null_vec = A.createVecLeft()   #for neumann problem
    # null_vec.set(1.0)
    # null_vec.normalize()
    # nullspace = PETSc.NullSpace().create(vectors=[null_vec])
    # A.setNullSpace(nullspace)

    return A, M



In [12]:
A, M = assemble_and_apply_bc(V, a, m)

In [13]:
print("A nnz:", A.getInfo())
print("M nnz:", M.getInfo())


A nnz: {'block_size': 1.0, 'nz_allocated': 32665.0, 'nz_used': 32665.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}
M nnz: {'block_size': 1.0, 'nz_allocated': 1784.0, 'nz_used': 1784.0, 'nz_unneeded': 0.0, 'memory': 0.0, 'assemblies': 1.0, 'mallocs': 0.0, 'fill_ratio_given': 0.0, 'fill_ratio_needed': 0.0, 'factor_mallocs': 0.0}


In [14]:
print(A.norm())
print(M.norm())

253.79529763546935
0.2975076060456055


setup eigenvalue solver

In [ ]:

def get_eigenvalues(A, M, n_eigen):

    eigensolver = SLEPc.EPS().create()
    eigensolver.setOperators(A, M)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    # eigensolver.setDimensions(nev=n_eigen, ncv=2*n_eigen)  # Larger subspace
    eigensolver.setDimensions(nev=n_eigen)  # Larger subspace
    # eigensolver.setTolerances(tol=1e-8, max_it=1000)
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

    eigensolver.setFromOptions()
    eigensolver.solve()
    n_conv = eigensolver.getConverged()

    if MPI.COMM_WORLD.rank == 0:
        print(f"Number of converged eigenvalues: {n_conv}")


    return eigensolver, n_conv

In [16]:
n_eigen = 5
eigensolver, n_conv = get_eigenvalues(A, M, n_eigen=n_eigen)

Error: error code 73
[0] EPSSolve() at /home/conda/feedstock_root/build_artifacts/slepc_1733219878637/work/src/eps/interface/epssolve.c:132
[0] EPSSetUp() at /home/conda/feedstock_root/build_artifacts/slepc_1733219878637/work/src/eps/interface/epssetup.c:394
[0] STSetUp() at /home/conda/feedstock_root/build_artifacts/slepc_1733219878637/work/src/sys/classes/st/interface/stsolve.c:546
[0] STSetUp_Shift() at /home/conda/feedstock_root/build_artifacts/slepc_1733219878637/work/src/sys/classes/st/impls/shift/shift.c:134
[0] KSPSetUp() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1741089330/work/src/ksp/ksp/interface/itfunc.c:427
[0] PCSetUp() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1741089330/work/src/ksp/pc/interface/precon.c:1086
[0] PCSetUp_LU() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1741089330/work/src/ksp/pc/impls/factor/lu/lu.c:87
[0] MatLUFactorSymbolic() at /home/conda/feedstock_root/build_artifact

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import dolfinx.plot

def plot_eigenfunctions(domain, V, eigensolver, A, nev_to_plot):
    """
    Plot the first few eigenfunctions for any given domain using matplotlib.
    Parameters:
        domain: dolfinx.mesh.Mesh
        V: dolfinx.fem.FunctionSpace
        eigensolver: SLEPc.EPS
        A: PETSc.Mat (needed for getVecs)
        n_conv: int (number of converged eigenpairs)
        nev_to_plot: int (number of eigenfunctions to plot)
    """
    if nev_to_plot == 0:
        return
    
    if MPI.COMM_WORLD.rank != 0:
        return

    topology, cell_types, geometry = plot.vtk_mesh(V)
    points = domain.geometry.x
    for i in range(nev_to_plot):
        eigval = eigensolver.getEigenvalue(i)
        error = eigensolver.computeError(i)

        print(f"Eigenvalue {i}: {eigval:.4f}  Znp = {np.sqrt(eigval):.2f} (Error: {error:.2e})")

        r, _ = A.getVecs()
        eigensolver.getEigenvector(i, r)

        uh = fem.Function(V)
        uh.x.petsc_vec.setArray(r.array)
        uh.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        # Create PyVista grid
        values = uh.x.array.real
        grid = pv.UnstructuredGrid(topology, cell_types, points)
        grid.point_data["u"] = values
        
        
        grid.set_active_scalars("u")
        warped = grid.warp_by_scalar()
        

         # Define the number of contour levels
        contour_levels = 25

        # Generate contours using the scalar data
        contours = grid.contour(contour_levels, scalars="u")
        # A plotting window is created with two sub-plots, one of the scalar
        # values and the other of the mesh is warped by the scalar values in
        # z-direction
        subplotter = pyvista.Plotter(shape=(1, 2))
        subplotter.subplot(0, 0)
        subplotter.add_text("Scalar contour field", font_size=14, color="black", position="upper_edge")
        subplotter.add_mesh(grid, show_scalar_bar=True)
        subplotter.add_mesh(contours, color="blue", line_width=1)

        subplotter.view_xy()

        subplotter.subplot(0, 1)
        subplotter.add_text("Warped function", position="upper_edge", font_size=14, color="black")
        subplotter.add_mesh(warped)
        subplotter.add_mesh(contours, color="blue", line_width=1)

        subplotter.show()

        # Plot
        # plotter = pv.Plotter(notebook=True)
        # plotter.add_mesh(grid scalars="u", cmap="rainbow")
        # plotter.view_xy()
        # plotter.add_text(f"Eigenfunction {i}, λ = {eigval:.2f}", font_size=12)
        # plotter.show()

# Usage example (after solving eigenproblem):
# plot_eigenfunctions(domain, V, eigensolver, A, n_conv, nev_to_plot=6)


In [ ]:
plot_eigenfunctions(domain, V, eigensolver, A, n_conv)